In [48]:
print("OK")

OK


In [19]:
%pwd

'c:\\Users\\jisna\\medical_chatbot\\End-to-End-Medical-Chatbot'

In [18]:
import os
os.chdir('../')


In [17]:
%pwd

'c:\\Users\\jisna\\medical_chatbot\\End-to-End-Medical-Chatbot\\research'

In [20]:


from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [21]:
from langchain.llms import CTransformers

In [22]:
PINECONE_API_KEY="pcsk_74RJkT_TFVcRGZFrzG69LPdMqdcWg9pPqHzuLw6vvTRPxjv5aDGR8Teqw1SzYQX67BgLxe"
PINECONE_API_ENV="us-east-1"

In [23]:
#Extract data from the pdf
def load_pdf(data):
     loader= DirectoryLoader(data,glob="*.pdf",
                    loader_cls=PyPDFLoader)
     documents=loader.load()
     return documents

     

In [24]:
extracted_data=load_pdf("data/")

In [25]:
# Create text chunks from loaded data
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    chunks=text_splitter.split_documents(extracted_data)
    return chunks

In [26]:
text_chunks=text_split(extracted_data)
print(f"Length of chunks: {len(text_chunks)}")

Length of chunks: 5860


In [27]:
from langchain.embeddings import HuggingFaceEmbeddings

In [28]:
def download_hugging_face_embedding():
    embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [29]:
embeddings=download_hugging_face_embedding()

C:\Users\jisna\AppData\Local\Temp\ipykernel_113068\22862419.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [14]:
query_result=embeddings.embed_query("Hello World")
print(f"Length of query result: {len(query_result)}")

Length of query result: 384


In [30]:
from dotenv import load_dotenv
load_dotenv()

True

In [31]:
PINECONE_API_KEY=os.environ.get("PINECONE_API_KEY")

In [32]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os
pc=Pinecone(api_key=PINECONE_API_KEY)
index_name="mchatbot"

In [33]:
if index_name not in pc.list_indexes().names():
   pc.create_index(
       name=index_name,
       dimensions=384,
       metric='cosine',
       spec=ServerlessSpec(cloud='aws',region='us-east-1')

 )

In [78]:
index=pc.Index(index_name)
index

In [34]:
import os
os.environ["PINECONE_API_KEY"]=PINECONE_API_KEY

In [22]:
#Embed each chunk and upsert the embeddings into your Pinecone Index
from langchain_pinecone import PineconeVectorStore
doc_search=PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings
)

In [83]:
doc_search

In [35]:
#load from existing index
from langchain_pinecone import PineconeVectorStore
doc_search=PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [36]:
retriever=doc_search.as_retriever(
                 search_type='similarity',search_kwargs={"k":3}

)

In [30]:
retrieved_docs=retriever.invoke("What is Acne?")

In [31]:
retrieved_docs

[Document(id='aea72780-d284-4e41-a888-00a471af9b82', metadata={'page': 39.0, 'source': 'data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='8309ffe4-f26b-4ea0-b575-e531c6b2545d', metadata={'page': 39.0, 'source': 'data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='aca5a49c-4385-4396-ad24-bca6c7a7fce2', metadata={'page': 38.0, 'source': 'data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.(Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25'),
 Document(id='f988984f-5272-4966-bd64-ae4062008619', metadata={'page': 38.0, 'source': 'data\\Medical_bo

In [37]:
from langchain.llms import CTransformers

In [38]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens': 500,
                          'temperature':0.4})

In [39]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an intelligent assistant specializing in answering questions based on provided context. "
    "Carefully read the retrieved docs and provide a concise, accurate response. "
    "If the retrieved docs do not contain the information needed to answer, you MUST respond with 'I don't know.' "
    "Do not attempt to predict or fabricate information. "
    "Keep your response to three sentences or fewer to maintain brevity."
    "\n\n"
    "Context:\n{context}"
)


Human_prompt="{input}"
prompt=ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human",Human_prompt)
    ]
)

In [40]:
question_answer_chain=create_stuff_documents_chain(llm,prompt)

In [41]:
rag_chain=create_retrieval_chain(retriever,question_answer_chain)

In [43]:
response=rag_chain.invoke({"input":"What is Acne ?"})
print(response['answer'])


Assistant: According to the Gale Encyclopedia of Medicine, acne is a skin disorder that occurs when the sebaceous glands become inflamed. It can affect anyone but is more common in women during puberty and in menstruating women. The exact cause of acne is not known, but it is believed to be related to hormonal changes in the body. (Sources: Gale Encyclopedia of Medicine, 2nd ed.)


In [43]:
response=rag_chain.invoke({"input":"Explain Stats?"})
print(response['answer'])

In [44]:
response

{'input': 'Explain Stats?',
 'context': [Document(id='8fd9ed31-fd90-45d6-a876-13acf2714891', metadata={'page': 98.0, 'source': 'data\\Medical_book.pdf'}, page_content='• CD4+ lymphocyte percentage. Some doctors think that\nthis is a more accurate test than the absolute count\nbecause the percentage does not depend on a manual\ncalculation of the number of types of different white\nblood cells. A white blood cell count that is broken\ndown into categories in this way is called a WBC dif-\nferential.\nIt is important for doctors treating AIDS patients to\nmeasure the lymphocyte count on a regular basis.\nExperts consulted by the United States Public Health'),
  Document(id='83171071-725f-4354-ab57-b2389cb110ec', metadata={'page': 98.0, 'source': 'data\\Medical_book.pdf'}, page_content='• CD4+ lymphocyte percentage. Some doctors think that\nthis is a more accurate test than the absolute count\nbecause the percentage does not depend on a manual\ncalculation of the number of types of differ

In [45]:
response=rag_chain.invoke({"input":"what is Calculus?"})
print(response['answer'])


```
Your response:
I don't know.


In [46]:
response=rag_chain.invoke({"input":"what is disease?"})
print(response['answer'])

Number of tokens (537) exceeded maximum context length (512).
Number of tokens (538) exceeded maximum context length (512).
Number of tokens (539) exceeded maximum context length (512).
Number of tokens (540) exceeded maximum context length (512).
Number of tokens (541) exceeded maximum context length (512).
Number of tokens (542) exceeded maximum context length (512).
Number of tokens (543) exceeded maximum context length (512).
Number of tokens (544) exceeded maximum context length (512).
Number of tokens (545) exceeded maximum context length (512).
Number of tokens (546) exceeded maximum context length (512).
Number of tokens (547) exceeded maximum context length (512).
Number of tokens (548) exceeded maximum context length (512).
Number of tokens (549) exceeded maximum context length (512).
Number of tokens (550) exceeded maximum context length (512).
Number of tokens (551) exceeded maximum context length (512).
Number of tokens (552) exceeded maximum context length (512).
Number o


Bartonella basillness are you have you have you think you have I have you have you have you have you have you have you have you have you have you may result from the patient sufferers are you might be associated with prominent symptoms do people infects can bears are you think you have you have you have you have you have you have you have you have you have you have I have you have you have you have you have you have you have you have you have you have you have you may result from the bartonella basillness does the patient sufferers are you have you have you think you might a person is caused by Bartonellosis do you have i have you have you have you may develops does bacteria or disrupts does Bartonella bas been associated with prominent symptoms does the patient has prominent symptoms do you want to you think you think you have you have you have you have you have you have I have you have you have you have you have you have you have you have you have you have you have you have you may 